In [4]:
import numpy as np
from nuscenes.nuscenes import NuScenes

def extract_point_clouds(nusc):
    # Initialize list to store point cloud datasets
    dataset = []

    for sample in nusc.sample:
        lidar_token = sample["data"]["LIDAR_TOP"]
        # Get lidar data path and boxes (we don't use boxes here)
        lidar_path, _, _ = nusc.get_sample_data(lidar_token)

        # Load lidar points
        pts = np.fromfile(lidar_path, dtype=np.float32, count=-1).reshape([-1, 5])
        
        # Depending on your specific need, you may want to transform or process pts here
        # For example, we can select only X, Y, Z position data
        pts_xyz = pts[:, :5     ]

        # Add the point cloud to the dataset
        dataset.append(pts_xyz)

    return dataset

# Usage example
# Initialize NuScenes
nusc = NuScenes(version='v1.0-mini', dataroot='../data/nuscenes', verbose=True)

# Extract dataset
point_cloud_dataset = extract_point_clouds(nusc)

# Now, `point_cloud_dataset` is a list where each element is a (N, 3) numpy array representing a point cloud.
for i, point_cloud in enumerate(point_cloud_dataset):
    print(f"Point cloud {i} has shape {point_cloud.shape}")


Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.596 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
Point cloud 0 has shape (34688, 5)
Point cloud 1 has shape (34720, 5)
Point cloud 2 has shape (34720, 5)
Point cloud 3 has shape (34688, 5)
Point cloud 4 has shape (34752, 5)
Point cloud 5 has shape (34784, 5)
Point cloud 6 has shape (34720, 5)
Point cloud 7 has shape (34720, 5)
Point cloud 8 has shape (34688, 5)
Point cloud 9 has shape (34688, 5)
Point cloud 10 has shape (34688, 5)
Point cloud 11 has shape (34720, 5)
Point cloud 12 has shape (34688, 5)
Point cloud 13 has shape (34752, 5)
Point cloud 14 has shape (34752, 5)
Point cloud 15 has shape (34784, 5)
Point cloud 16 has shape (34784, 5)
Point cloud 17 has shape (34752,

In [6]:
# get max of points clouds
max_points = 0
for i, point_cloud in enumerate(point_cloud_dataset):
    if point_cloud.shape[0] > max_points:
        max_points = point_cloud.shape[0]
max_points

34816

In [1]:
import torch

In [8]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from nuscenes.nuscenes import NuScenes

class NuScenesLidarDataset(Dataset):
    def __init__(self, nuscenes_path, max_points=34816, version='v1.0-mini', lidarseg_path='lidarseg'):
        self.nusc = NuScenes(version=version, dataroot=nuscenes_path, verbose=True)
        self.lidarseg_path = os.path.join(nuscenes_path, lidarseg_path, version)
        self.max_points = max_points

    def __len__(self):
        return len(self.nusc.sample)

    def __getitem__(self, idx):
        sample = self.nusc.sample[idx]
        lidar_token = sample['data']['LIDAR_TOP']
        lidar_data = self.nusc.get('sample_data', lidar_token)
        lidar_filepath = os.path.join(self.nusc.dataroot, lidar_data['filename'])
        lidar_points = np.fromfile(lidar_filepath, dtype=np.float32).reshape(-1, 5)

        # Load corresponding segmentation labels
        lidarseg_filepath = os.path.join(self.lidarseg_path, lidar_token + '_lidarseg.bin')
        if os.path.exists(lidarseg_filepath):
            labels = np.fromfile(lidarseg_filepath, dtype=np.uint8)
        else:
            labels = np.zeros(len(lidar_points), dtype=np.uint8)

        # Pad or truncate the point clouds and labels
        num_points = len(lidar_points)
        if num_points < self.max_points:
            pad_size = self.max_points - num_points
            lidar_points = np.pad(lidar_points, ((0, pad_size), (0, 0)), mode='constant', constant_values=0)
            labels = np.pad(labels, (0, pad_size), mode='constant', constant_values=255)  # Padding labels with a null value (e.g., 255)
        elif num_points > self.max_points:
            lidar_points = lidar_points[:self.max_points]
            labels = labels[:self.max_points]

        # Convert to PyTorch tensors
        lidar_points_tensor = torch.from_numpy(lidar_points).float()
        labels_tensor = torch.from_numpy(labels).long()

        return lidar_points_tensor, labels_tensor


# Usage
nuscenes_path = '../data/nuscenes'
dataset = NuScenesLidarDataset(nuscenes_path)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Example of iterating through the DataLoader
for lidar_points, labels in dataloader:
    # Process the batch of LiDAR points and labelsç
    pass
print(lidar_points.shape)
print(labels.shape)


Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.418 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
torch.Size([4, 34816, 5])
torch.Size([4, 34816])


In [10]:
# get number of unique labels
unique_labels = set()
for i, point_cloud in enumerate(point_cloud_dataset):
    unique_labels.update(point_cloud[:, 4])
print(unique_labels)

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0}


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TNet(nn.Module):
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.fc3.bias.data.fill_(0)
        self.fc3.weight.data.fill_(0)

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = torch.eye(self.k).repeat(batchsize, 1, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x.view(-1, self.k, self.k) + iden
        return x

class PointNetSeg(nn.Module):
    def __init__(self, num_classes):
        super(PointNetSeg, self).__init__()
        self.tnet = TNet(k=5)

        self.conv1 = nn.Conv1d(5, 64, 1)
        self.bn1 = nn.BatchNorm1d(64)

        self.conv2 = nn.Conv1d(64, 128, 1)
        self.bn2 = nn.BatchNorm1d(128)

        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.bn3 = nn.BatchNorm1d(1024)

        # Decoder
        self.conv4 = nn.Conv1d(1088, 512, 1)  # Concatenated feature size: 1024 + 64
        self.bn4 = nn.BatchNorm1d(512)

        self.conv5 = nn.Conv1d(512, 256, 1)
        self.bn5 = nn.BatchNorm1d(256)

        self.conv6 = nn.Conv1d(256, num_classes, 1)

    def forward(self, x):
        batchsize = x.size()[0]
        num_points = x.size()[2]

        # Encoder
        trans = self.tnet(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)

        x1 = F.relu(self.bn1(self.conv1(x)))
        x2 = F.relu(self.bn2(self.conv2(x1)))

        x3 = self.bn3(self.conv3(x2))
        x3 = torch.max(x3, 2, keepdim=True)[0]
        x3 = x3.repeat(1, 1, num_points)

        # Concatenate global and local features
        x_concat = torch.cat([x3, x1], 1)

        # Decoder
        x = F.relu(self.bn4(self.conv4(x_concat)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.conv6(x)

        return x.transpose(2, 1)  # Transpose to get (B, N, num_classes)

In [21]:
model = PointNetSeg(num_classes=32)
for lidar_points, labels in dataloader:
    # Process the batch of LiDAR points and labelsç
    pass
lidar_points = lidar_points.transpose(1, 2) 
output = model(lidar_points)
print(lidar_points.shape)
print(output.shape)
print(labels.shape)

torch.Size([4, 5, 34816])
torch.Size([4, 34816, 32])
torch.Size([4, 34816])
